In [35]:
import pandas as pd
import numpy as np
import os
import yfinance as yf


In [44]:
## Read in files to get clean base dataframe
directory = "Data"
df_list = []

for root, dirs, files in os.walk(directory):
    if "INFOTABLE.tsv" in files and "OTHERMANAGER2.tsv" in files and "OTHERMANAGER.tsv" in files:
        intotable_path = os.path.join(root, "INFOTABLE.tsv")
        manager1_path = os.path.join(root, "OTHERMANAGER.tsv")
        manager2_path = os.path.join(root, "OTHERMANAGER2.tsv")
        manager1_df = pd.read_csv(manager1_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])
        manager2_df = pd.read_csv(manager2_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])

        manager_df = pd.concat([manager1_df, manager2_df], ignore_index=True)
        manager_df.rename(columns={'NAME':'FUND'}, inplace=True)

        infotable_df = pd.read_csv(intotable_path, sep = "\t", 
                                   usecols=["ACCESSION_NUMBER","NAMEOFISSUER","TITLEOFCLASS",
                                            "CUSIP", "VALUE", "SSHPRNAMT"])
        # Add date
        sub_dir = os.path.basename(root)
        datetime = sub_dir.split("-")[1].split("_")[0]
        infotable_df.insert(0, "Datetime", datetime)
        infotable_df["Datetime"] = pd.to_datetime(infotable_df["Datetime"])
        
        merged_df = pd.merge(infotable_df, manager_df, on="ACCESSION_NUMBER", how="inner") #fund name non-optional
        merged_df.drop(columns=["ACCESSION_NUMBER"], inplace=True)
        df_list.append(merged_df)

df_full = pd.concat(df_list, ignore_index=True)


## 

In [46]:
df_full

,Datetime,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,FUND
0,2024-05-31,CREDICORP LTD,COM,G2519Y108,757896,5055,METLIFE INC
1,2024-05-31,PAGSEGURO DIGITAL LTD,COM CL A,G68707101,119700,9599,METLIFE INC
2,2024-05-31,XP INC,CL A,G98239109,48438,1858,METLIFE INC
3,2024-05-31,INTERCORP FINL SVCS INC,SHS,P5626F128,1336645,60895,METLIFE INC
4,2024-05-31,COPA HOLDINGS SA,CL A,P31076105,1680442,15807,METLIFE INC
...,...,...,...,...,...,...,...
35529354,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,BANK VONTOBEL AG
35529355,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Swiss Financial Advisers AG
35529356,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Bank Vontobel Europe AG
35529357,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Asset Management Ltd


# Ticker
Wharton Research Data Services. "WRDS" wrds.wharton.upenn.edu, accessed 2024-11-30.



In [104]:
cusip_mapping_df = pd.read_csv("Data/wrld_cusip.csv")
# remove no ticker
cusip_mapping_df = cusip_mapping_df.dropna(subset=['TICKER'])
cusip_mapping_df = cusip_mapping_df[['TICKER', 'COMNAM', 'CUSIP']]
# drop duplicats for faster merging
cusip_mapping_df = cusip_mapping_df.drop_duplicates(subset=['CUSIP'], keep='first')
cusip_mapping_df.columns = ["TICKER", "COMNAM", "map_cusip"]


In [105]:
cusip_mapping_df

,TICKER,COMNAM,map_cusip
0,JJSF,J & J SNACK FOODS CORP,46603210
60,DGSE,D G S E COMPANIES INC,29402E10
120,PLXS,PLEXUS CORP,72913210
180,RMCF,ROCKY MOUNTAIN CHOC FAC INC NEW,77467X10
240,HNGR,HANGER INC,41043F20
...,...,...,...
522599,VPG,VISHAY PRECISION GROUP INC,92835K10
522659,FN,FABRINET,G3323L10
522719,CBOE,C B O E GLOBAL MARKETS INC,12503M10
522779,SANW,S & W SEED CO,78513510


## First round of ticker merge

In [63]:
# remove last digit of cusip in the 13f data
df_full['map_cusip'] = df_full['CUSIP'].str[:-1]


In [106]:
# merge
merged_df = df_full.merge(cusip_mapping_df, on='map_cusip', how='left')


## Secondary merge

In [110]:
missing_mapping = merged_df[merged_df['TICKER'].isna()]

# drop duplicated COMNAM column
short_cusip_mapping_df = cusip_mapping_df.drop_duplicates(subset=['COMNAM'], keep='first')

secondary_merge = missing_mapping.merge(
    short_cusip_mapping_df,
    left_on='NAMEOFISSUER',
    right_on='COMNAM',
    how='left',
    suffixes=('', '_secondary')
)


In [117]:
# Number of columns that has value for secondary merge
secondary_merge[secondary_merge['TICKER_secondary'].notna()]

,Datetime,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,FUND,map_cusip,TICKER,COMNAM,TICKER_secondary,COMNAM_secondary,map_cusip_secondary
0,2024-05-31,BANCO BRADESCO S A,SP ADR PFD NEW,059460303,211082,60309,METLIFE INC,05946030,NaN,NaN,BBDO,BANCO BRADESCO S A,05946040
138,2024-05-31,LIBERTY BROADBAND CORP,COM,530307503,33,1404,TODD INTERNATL INTRINSIC VALUE,53030750,NaN,NaN,LBRDA,LIBERTY BROADBAND CORP,53030710
139,2024-05-31,LIBERTY BROADBAND CORP,COM,530307503,33,1404,TODD INTL INTRINSIC VALUE OPPORTUNITY,53030750,NaN,NaN,LBRDA,LIBERTY BROADBAND CORP,53030710
140,2024-05-31,LIBERTY BROADBAND CORP,COM,530307503,33,1404,MED S&P 600 STOCK INDEX,53030750,NaN,NaN,LBRDA,LIBERTY BROADBAND CORP,53030710
141,2024-05-31,LIBERTY BROADBAND CORP,COM,530307503,33,1404,UBS VALUE ORIENTED,53030750,NaN,NaN,LBRDA,LIBERTY BROADBAND CORP,53030710
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038567,2024-08-31,BANCO BRADESCO S A,SP ADR PFD NEW,059460303,13778657,3936759,BANK VONTOBEL AG,05946030,NaN,NaN,BBDO,BANCO BRADESCO S A,05946040
1038568,2024-08-31,BANCO BRADESCO S A,SP ADR PFD NEW,059460303,13778657,3936759,Vontobel Swiss Financial Advisers AG,05946030,NaN,NaN,BBDO,BANCO BRADESCO S A,05946040
1038569,2024-08-31,BANCO BRADESCO S A,SP ADR PFD NEW,059460303,13778657,3936759,Bank Vontobel Europe AG,05946030,NaN,NaN,BBDO,BANCO BRADESCO S A,05946040
1038570,2024-08-31,BANCO BRADESCO S A,SP ADR PFD NEW,059460303,13778657,3936759,Vontobel Asset Management Ltd,05946030,NaN,NaN,BBDO,BANCO BRADESCO S A,05946040


In [120]:
# align index for secondary merge
secondary_merge.index = merged_df.loc[merged_df['TICKER'].isna()].index

merged_df.loc[merged_df['TICKER'].isna(), 'TICKER'] = secondary_merge['TICKER_secondary']


In [122]:
# show most tickers were found
merged_df[merged_df['TICKER'].notna()]

,Datetime,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,FUND,map_cusip,TICKER,COMNAM
0,2024-05-31,CREDICORP LTD,COM,G2519Y108,757896,5055,METLIFE INC,G2519Y10,BAP,CREDICORP LTD
1,2024-05-31,PAGSEGURO DIGITAL LTD,COM CL A,G68707101,119700,9599,METLIFE INC,G6870710,PAGS,PAGSEGURO DIGITAL LTD
2,2024-05-31,XP INC,CL A,G98239109,48438,1858,METLIFE INC,G9823910,XP,X P INC
3,2024-05-31,INTERCORP FINL SVCS INC,SHS,P5626F128,1336645,60895,METLIFE INC,P5626F12,IFS,INTERCORP FINANCIAL SERVICES INC
4,2024-05-31,COPA HOLDINGS SA,CL A,P31076105,1680442,15807,METLIFE INC,P3107610,CPA,COPA HOLDINGS SA
...,...,...,...,...,...,...,...,...,...,...
35529354,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,BANK VONTOBEL AG,88355610,TMO,THERMO FISHER SCIENTIFIC INC
35529355,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Swiss Financial Advisers AG,88355610,TMO,THERMO FISHER SCIENTIFIC INC
35529356,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Bank Vontobel Europe AG,88355610,TMO,THERMO FISHER SCIENTIFIC INC
35529357,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Asset Management Ltd,88355610,TMO,THERMO FISHER SCIENTIFIC INC


# Yfinance

In [134]:
unique_tickers = merged_df['TICKER'].dropna().unique()

# date range based on the datetime index in df_full
lowest_date = merged_df['Datetime'].min().date()
highest_date = merged_df['Datetime'].max().date()


In [135]:
lowest_date

datetime.date(2024, 5, 31)

In [125]:
len(unique_tickers)

10634

In [136]:
ticker = unique_tickers[0]

yf.download(ticker,start=str(lowest_date), end=str(highest_date))

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BAP,BAP,BAP,BAP,BAP,BAP
Date,,,,,,
2024-05-31,162.645767,165.320007,165.509995,163.020004,163.169998,430100
2024-06-03,162.458832,165.130005,166.679993,162.860001,166.539993,198800
2024-06-04,158.513687,161.119995,164.339996,160.809998,163.259995,131900
2024-06-05,159.930405,162.559998,164.229996,161.509995,161.509995,246600
2024-06-06,160.432159,163.070007,164.110001,162.020004,162.910004,160700
...,...,...,...,...,...,...
2024-08-26,166.541702,169.279999,170.750000,168.500000,170.100006,163700
2024-08-27,166.138321,168.869995,169.940002,167.800003,168.970001,71800


In [137]:
# get stock data
stock_data = []

for ticker in unique_tickers:
    try:
        ticker_data = yf.Ticker(ticker)
        history = yf.download(ticker,start=str(lowest_date), end=str(highest_date))

        if not history.empty:
            for date, row in history.iterrows():
                stock_data.append({
                    'Ticker': ticker,
                    'Date': date,
                    'Close': row['Close'],
                    'Volume': row['Volume'],
                    # Extract other metrics (if available)
                    'Market Cap': ticker_data.info.get('marketCap', None),
                    'PE Ratio': ticker_data.info.get('forwardPE', None)
                })
    except Exception as e:
        print(f"Error fetching data for ticker {ticker}: {e}")

# into a df
stock_data_df = pd.DataFrame(stock_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Error fetching data for ticker EW: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Error fetching data for ticker ELF: Expecting value: line 1 column 1 (char 0)


Failed to get ticker 'EXP' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EXP']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'EHC' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EHC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'NVST' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVST']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'EQH' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EQH']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'ETRN

...

Failed to get ticker 'BCIM' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BCIM']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'PMGM' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['PMGM']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'ATLX' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ATLX']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'ATS' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ATS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '

Error fetching data for ticker STBX: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


Failed to get ticker 'BSVN' reason: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BSVN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CAP']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MUDS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['HYW']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[************

Error fetching data for ticker EDTK: Expecting value: line 1 column 1 (char 0)


Failed to get ticker 'JZXN' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['JZXN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'MNDO' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MNDO']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'PCGG' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['PCGG']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'IRBA' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['IRBA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker

# AUM

In [26]:
# Join with aum 
aum_df = pd.read_csv("Data/aum_df")
df_full = df_full.merge(aum_df, left_on='FUND', right_on="fund_name", how='left')

In [27]:
# Set multiindex fund-level and datetime level
df_full.drop("fund_name", inplace=True, axis=1)
df_full.set_index(["FUND", "Datetime"], inplace=True)

In [28]:
df_full

NAMEOFISSUER  \
FUND                                 Datetime                                   
METLIFE INC                          2024-05-31                 CREDICORP LTD   
                                     2024-05-31         PAGSEGURO DIGITAL LTD   
                                     2024-05-31                        XP INC   
                                     2024-05-31       INTERCORP FINL SVCS INC   
                                     2024-05-31              COPA HOLDINGS SA   
...                                                                       ...   
BANK VONTOBEL AG                     2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Swiss Financial Advisers AG 2024-08-31  THERMO FISHER SCIENTIFIC INC   
Bank Vontobel Europe AG              2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management Ltd        2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management S.A.       2024-08-31  THERMO FISHER SCIENTIFIC INC   

                                                TITLEOFCLASS    CUSIP_x  \
FUND                                 Datetime                             
METLIFE INC                          2024-05-31          COM  G2519Y108   
                                     2024-05-31     COM CL A  G68707101   
                                     2024-05-31         CL A  G98239109   
                                     2024-05-31          SHS  P5626F128   
                                     2024-05-31         CL A  P31076105   
...                                                      ...        ...   
BANK VONTOBEL AG                     2024-08-31          COM  883556102   
Vontobel Swiss Financial Advisers AG 2024-08-31          COM  883556102   
Bank Vontobel Europe AG              2024-08-31          COM  883556102   
Vontobel Asset Management Ltd        2024-08-31          COM  883556102   
Vontobel Asset Management S.A.       2024-08-31          COM  883556102   

                                                   VALUE  SSHPRNAMT map_cusip  \
FUND                                 Datetime                                   
METLIFE INC                          2024-05-31   757896       5055  G2519Y10   
                                     2024-05-31   119700       9599  G6870710   
                                     2024-05-31    48438       1858  G9823910   
                                     2024-05-31  1336645      60895  P5626F12   
                                     2024-05-31  1680442      15807  P3107610   
...                                                  ...        ...       ...   
BANK VONTOBEL AG                     2024-08-31   207539        391  88355610   
Vontobel Swiss Financial Advisers AG 2024-08-31   207539        391  88355610   
Bank Vontobel Europe AG              2024-08-31   207539        391  88355610   
Vontobel Asset Management Ltd        2024-08-31   207539        391  88355610   
Vontobel Asset Management S.A.       2024-08-31   207539        391  88355610   

                                                TICKER  \
FUND                                 Datetime            
METLIFE INC                          2024-05-31    BAP   
                                     2024-05-31   PAGS   
                                     2024-05-31     XP   
                                     2024-05-31    IFS   
                                     2024-05-31    CPA   
...                                                ...   
BANK VONTOBEL AG                     2024-08-31    TMO   
Vontobel Swiss Financial Advisers AG 2024-08-31    TMO   
Bank Vontobel Europe AG              2024-08-31    TMO   
Vontobel Asset Management Ltd        2024-08-31    TMO   
Vontobel Asset Management S.A.       2024-08-31    TMO   

                                                                           COMNAM  \
FUND                                 Datetime                                       
METLIFE INC                          2024-05-31         

In [29]:
# ADD RANK COLUMN
df_full['RANK'] = df_full.groupby(['FUND', 'Datetime'])['VALUE'].rank(ascending=False, method = 'dense')

In [30]:
# ADD columns
df_full['TotalHoldingsMarketValue'] = df_full.groupby(['FUND', 'Datetime'])['VALUE'].transform('sum')
df_full['Percentage'] = (df_full['VALUE'] / df_full['TotalHoldingsMarketValue']) * 100
# add change in percentage
df_full['Previous Percentage'] = df_full.groupby('FUND')['Percentage'].shift(1)
df_full['ChangeInPercentage'] = df_full['Previous Percentage'] - df_full['Percentage']

In [31]:
# Add AUM from results from test_aum


In [32]:
# revome if no shares
df_full = df_full[df_full["SSHPRNAMT"] != 0]

In [33]:
# Number of share to matter
n = 10000 # we are going to ignore holdings of less than 5000 shares
df_full = df_full[df_full["SSHPRNAMT"] > n]

# find unique cusips to download from yfinance

unique_cusips = df_full["CUSIP"].unique()

KeyError: 'CUSIP'

In [12]:
len(unique_cusips)

18435

In [24]:
date = '2024-08-31'
df_full.loc["METLIFE INC"].sort_values("RANK")

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,aum,aum_date,RANK,TotalHoldingsMarketValue,Percentage,Previous Percentage,ChangeInPercentage
Datetime,,,,,,,,,,,,
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,921469348,8406800,NaN,NaN,1.0,2.184377e+11,4.218455e-01,0.020864,-0.400981
2024-08-31,MICROSOFT CORP,COM,594918104,1040306071,2327567,NaN,NaN,1.0,2.111874e+10,4.925985e+00,0.037682,-4.888303
2024-08-31,APPLE INC,COM,037833100,952286948,4521351,NaN,NaN,2.0,2.111874e+10,4.509203e+00,0.003477,-4.505726
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,893382191,8473700,NaN,NaN,2.0,2.184377e+11,4.089873e-01,0.016460,-0.392528
2024-05-31,MICROSOFT CORPORATION,COM,594918104,861591226,2047897,NaN,NaN,3.0,2.184377e+11,3.944335e-01,0.003604,-0.390830
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,ZIOPHARM ONCOLOGY INC,COM,98973P101,13190,12101,NaN,NaN,43759.0,2.184377e+11,6.038336e-06,0.001000,0.000994
2024-05-31,MULTIPLAN CORP,COM,62548M100,10844,13368,NaN,NaN,43850.0,2.184377e+11,4.964345e-06,0.000121,0.000116
2024-05-31,VBI VACCINES INC CDA,COM NEW,91822J103,10050,12431,NaN,NaN,43875.0,2.184377e+11,4.600855e-06,0.000274,0.000270


In [25]:
df_full.loc["METLIFE INC"].sort_values("Percentage",ascending=False)

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,aum,aum_date,RANK,TotalHoldingsMarketValue,Percentage,Previous Percentage,ChangeInPercentage
Datetime,,,,,,,,,,,,
2024-08-31,MICROSOFT CORP,COM,594918104,1040306071,2327567,NaN,NaN,1.0,2.111874e+10,4.925985e+00,0.037682,-4.888303
2024-08-31,APPLE INC,COM,037833100,952286948,4521351,NaN,NaN,2.0,2.111874e+10,4.509203e+00,0.003477,-4.505726
2024-08-31,NVIDIA CORP,COM,67066G104,945075811,7649958,NaN,NaN,3.0,2.111874e+10,4.475057e+00,0.002031,-4.473026
2024-08-31,AMAZON COM INC,COM,023135106,555798595,2876060,NaN,NaN,4.0,2.111874e+10,2.631779e+00,0.001350,-2.630428
2024-08-31,SPDR S&P 500 ETF TR TR UNIT,TR UNIT,78462F103,506156511,966890,NaN,NaN,5.0,2.111874e+10,2.396717e+00,0.000648,-2.396069
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,ZIOPHARM ONCOLOGY INC,COM,98973P101,13190,12101,NaN,NaN,43759.0,2.184377e+11,6.038336e-06,0.001000,0.000994
2024-05-31,MULTIPLAN CORP,COM,62548M100,10844,13368,NaN,NaN,43850.0,2.184377e+11,4.964345e-06,0.000121,0.000116
2024-05-31,VBI VACCINES INC CDA,COM NEW,91822J103,10050,12431,NaN,NaN,43875.0,2.184377e+11,4.600855e-06,0.000274,0.000270


In [26]:
df_full.index.value_counts()

FUND                                  Datetime  
PARAMETRIC PORTFOLIO ASSOCIATES LLC   2024-08-31    95145
GOLDMAN SACHS ASSET MANAGEMENT, L.P.  2024-05-31    94380
AYCO CO L P                           2024-05-31    93094
GOLDMAN SACHS & CO. LLC               2024-05-31    93094
GOLDMAN SACHS BANK AG                 2024-05-31    93094
                                                    ...  
TimesSquare Capital Management, LLC   2024-05-31        1
                                      2024-08-31        1
Foundry Group, LLC                    2024-08-31        1
                                      2024-05-31        1
 Burgundy Asset Management Ltd.       2024-08-31        1
Name: count, Length: 6808, dtype: int64